In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage , AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import yaml
from yaml.loader import SafeLoader
from RAG import return_rag_chain

from elevenlabs.client import ElevenLabs
from elevenlabs import play
import os


AttributeError: partially initialized module 'whisper' has no attribute 'load_model' (most likely due to a circular import)

In [1]:
import whisper


In [2]:
model = whisper.load_model("base.en")  # Options: "tiny", "base", "small", "medium", "large"

def transcribe_long_audio(file_path):
    result = model.transcribe(file_path)
    return result["text"]

# Example usage
audio_file = "Record (online-voice-recorder.com).mp3"
transcribed_text = transcribe_long_audio(audio_file)
print("Transcribed Text:", transcribed_text)

c:\Users\Hossa\miniconda3\envs\py310\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device

Transcribed Text:  Hello, can you tell me what is the best source of protein in order to increase my muscles?


In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [2]:
from faster_whisper import WhisperModel

model_size = "small"
model = WhisperModel(model_size, device="cuda")


In [5]:

# Run on GPU with FP16

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe("Record (online-voice-recorder.com).mp3", language="en",beam_size=5)

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

[0.00s -> 6.00s]  Hello, can you tell me what is the best source of protein in order to increase my muscles?


In [2]:
print(whisper.List)

NameError: name 'whisper' is not defined

In [24]:
from elevenlabs import  stream


In [2]:
with open('config.yaml') as file:
    config = yaml.load(file, Loader=SafeLoader)
elevenlabs_api_key = config['elevnlabs']['apiKey']

In [26]:
from elevenlabs import generate

ImportError: cannot import name 'generate' from 'elevenlabs' (c:\Users\Hossa\miniconda3\envs\py310\lib\site-packages\elevenlabs\__init__.py)

In [3]:
os.environ["ELEVENLABS_API_KEY"] = elevenlabs_api_key

In [4]:
client = ElevenLabs(api_key=elevenlabs_api_key)

In [10]:
rag=return_rag_chain()

Vectorstore created successfully


In [ ]:
response=rag.invoke({"input": "tell what is the best way to increase my gym session time as i get tired so fast.", "chat_history": []})


AttributeError: 'RunnableSequence' object has no attribute 'pick_chain'

In [32]:
def print_responses(responses):
    for response in responses:
        print(response, end="",flush=True)

In [28]:

rag_chain=rag.pick('answer')

In [31]:
response=rag_chain.stream({"input": "tell what is the best way to increase my gym session time as i get tired so fast.", "chat_history": []})

In [41]:
question = input("Ask a question: ")
response = rag.invoke({"input": question, "chat_history": []})["answer"]

audio = client.text_to_speech.convert(
    text=response,
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_flash_v2",
    output_format="mp3_44100_128",
)
play(audio)

